In [45]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [46]:
print_to_excel = True

Загрузка файлов

In [47]:
# MANUAL VARIABLES
BRFHRG_len = 567
BRFTO_len = 236
BRSFT_len = 206

sheet_name = 'Balance Sheet'
day = '12.11.2024'
files_list = [x for x in os.listdir() if 'Exposure' in x and day in x and '~' not in x]

# reading files
files_df = pd.DataFrame()
for file in files_list:
    
    if 'BRFHRG' in file:
        header = 0
        usecols = [0,1,2,3,4,5,6,7,8]
        data_len = BRFHRG_len
    elif 'BRFTO' in file:
        header = 0
        usecols = [0,1,2,3,4,5,6,7,8]
        data_len = BRFTO_len
    elif 'BRSFT' in file:
        header = 1
        usecols = [0,1,2,3,4,5,6]
        data_len = BRSFT_len
    
    file_data = pd.read_excel(file, sheet_name=sheet_name, header=header, usecols=usecols)
    file_data = file_data[~file_data['Seg Conta'].isna()] # del empty rows in seg conta
    # del not int form seg conta
    file_data['Seg Conta'] = file_data['Seg Conta'].astype(int, errors='ignore')
    file_data = file_data[file_data['Seg Conta'].apply(lambda x: str(type(x)))=='<class \'int\'>']
    if len(file_data) == data_len:
        file_data['sourse'] = file
        files_df = pd.concat([files_df,file_data])
        continue
    else:
        print('problem with len in ', file)
        print('data len is', len(file_data))
        print('need len is', data_len)

assert len(files_df) == BRFHRG_len + BRFTO_len + BRSFT_len
len(files_df)

1009

In [48]:
# reading En file
EN_desc_data = pd.read_excel('Full_EN_desc.xlsx')
EN_desc_data['Seg Conta'] = EN_desc_data['Seg Conta'].astype(int, errors='ignore')

# in OCP or not 
Balance_Name_list = files_df['Balance Name'].unique().tolist()
in_OCP_list = ['Trade Receivables', 'Trade receivables','Cash and Applications','Other receivables', 'Other payables','Trade payable','Borrowings Third Parties','Advances from customers', 'Related parties - customers BRL','Related parties - loan agreement', 'Trade payables - credit assignment Domestic','Related parties - trade payables BRL','Trade payables - credit assignment - International','Related parties - trade payables USD']
not_in_OCP_list = [x for x in Balance_Name_list if x not in in_OCP_list]
in_ocp_df = pd.concat([pd.DataFrame({'Balance_Name_in_OCP':in_OCP_list}),pd.DataFrame({'Balance_Name_NOT_in_OCP':not_in_OCP_list})], axis=1)

files_df['In_OCP'] = False
files_df.loc[files_df['Balance Name'].isin(in_OCP_list), 'In_OCP'] = True

# drop duplicates in files data
files_df_drop_duples = files_df.drop_duplicates(subset=['Seg Conta','Currency Denomination','Balance Name','Class Account','Class Account 2','Account Description (English)','More Comments','sourse','In_OCP'])
print(len(files_df_drop_duples))

# merge EN on 2 cols
merge_data = files_df_drop_duples.merge(EN_desc_data, on=['Seg Conta','Descr Conta'], how='left', suffixes=['','_map'])
merge_data_full = merge_data[~merge_data['Account Description (English)_map'].isna()]
merge_data_full_len = len(merge_data_full)

# merge EN on 1 col
empty_data = merge_data.loc[merge_data['Account Description (English)_map'].isna(),:'In_OCP']
merge_data_empty = empty_data.merge(EN_desc_data, on='Descr Conta', how='left', suffixes=['','_map'])
merge_data_empty = merge_data_empty.drop_duplicates(subset='Seg Conta')
new_data_after_empty = merge_data_empty[~merge_data_empty['Account Description (English)_map'].isna()]
# concat
merge_data_full = pd.concat([merge_data_full,new_data_after_empty])
assert len(merge_data_full) == merge_data_full_len + len(new_data_after_empty)

# equal of columns
merge_data_full['CA_equal'] = merge_data_full['Class Account'].str.lower() == merge_data_full['Class Account_map'].str.lower()
merge_data_full['CA2_equal'] = merge_data_full['Class Account 2'].str.lower() == merge_data_full['Class Account 2_map'].str.lower()

# translate
new_empty_data = empty_data[~empty_data['Seg Conta'].isin(new_data_after_empty['Seg Conta'].tolist())]
print(len(empty_data), len(new_empty_data))


973
57 55


In [49]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_BRL_Balance_sheet_translate.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
        in_ocp_df.to_excel(writer, index=False, sheet_name='in_OCP_atributs')
        merge_data_full.to_excel(writer, index=False, sheet_name='translated_data')
        new_empty_data.to_excel(writer, index=False, sheet_name='need_to_translate')
        EN_desc_data.to_excel(writer, index=False, sheet_name='EN_map')


Что входить в расчет

In [53]:
# files_df.shape[0]
# Balance_Name_list = files_df['Balance Name'].unique().tolist()
# in_OCP_list = ['Trade Receivables', 'Trade receivables','Cash and Applications','Other receivables', 'Other payables','Trade payable','Borrowings Third Parties','Advances from customers', 'Related parties - customers BRL','Related parties - loan agreement', 'Trade payables - credit assignment Domestic','Related parties - trade payables BRL','Trade payables - credit assignment - International','Related parties - trade payables USD']
# not_in_OCP_list = [x for x in Balance_Name_list if x not in in_OCP_list]

# # files_df[files_df['Balance Name'].isin(not_in_OCP_list)]
# not_in_OCP_list
# apv_data = files_df[files_df['Balance Name'].isin(['Derivatives Results and APV'])]
files_df[files_df['Descr Conta'].str.contains('IFRS')].shape[0]

14

In [51]:
# files_df['Saldo_Final_abs'] = files_df['Saldo Final'].abs()
# files_df[(files_df.duplicated(subset='Saldo_Final_abs',keep=False))&(files_df['Saldo_Final_abs']>0)].sort_values('Saldo_Final_abs')
# apv_data[apv_data['Saldo_Final_abs']>5*10**6]

apv_data

,Seg Conta,Descr Conta,Saldo Final,Currency Denomination,Balance Name,Class Account,Class Account 2,Account Description (English),More Comments,sourse,In_OCP
86,69997,HEDGE MTM a receber,0.000000e+00,"BRL, out of OCP",Derivatives Results and APV,Assets,Current assets,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
87,69998,HEDGE a receber,0.000000e+00,"BRL, out of OCP",Derivatives Results and APV,Assets,Current assets,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
113,206990,AVP contas a receber nacional,-5.264808e+06,"BRL, out of OCP",Derivatives Results and APV,Assets,Current assets,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
348,400010,AVP Fornec Nac,1.095378e+05,"BRL, out of OCP",Derivatives Results and APV,Liabilities,Current Liabilities,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
351,500502,Ajuste variação cambial contas a pagar,0.000000e+00,"BRL, out of OCP",Derivatives Results and APV,Liabilities,Current Liabilities,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
355,500512,(-) AVP contas a pagar arrendamentos IFRS 16,-2.121277e+05,"BRL, out of OCP",Derivatives Results and APV,Liabilities,Current Liabilities,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
361,420990,AVP Fornec Exter,3.629328e+06,"BRL, out of OCP",Derivatives Results and APV,Liabilities,Current Liabilities,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
362,420999,Ajuste variacao cambial fornecedores estrangeiros,0.000000e+00,"BRL, out of OCP",Derivatives Results and APV,Liabilities,Current Liabilities,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
428,459900,HEDGE a pagar,0.000000e+00,"BRL, out of OCP",Derivatives Results and APV,Liabilities,Current Liabilities,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
429,459902,HEDGE MTM a pagar,0.000000e+00,"BRL, out of OCP",Derivatives Results and APV,Liabilities,Current Liabilities,NaN,NaN,BRFHRG FX Exposure 12.11.2024 - With pnl Tax ...,False
